In [9]:
import numpy as np
import torch

MLPs have two hidden layers
ReLU activation for hidden layers, Sigmoid for last layer

for f, each hidden layer has 256 nodes
for g, each hidden layer has 5000 nodes

Train with Adam optimizer.

learning schedule step-wise decay:
for f: start with 1e-4, then every 20 epochs, reduce it by 30%. Total 100 epochs.
for g: start with 1e-5, then every 200 epochs, reduce it by 30%. Total 600 epochs.